### Programming for Science and Finance

*Prof. Götz Pfeiffer, School of Mathematical and Statistical Sciences, University of Galway*

# Notebook 4: Programming with Objects II

This notebook accompanies **Part I**. You will:

* Deepen your understanding of **object-oriented programming (OOP)** by building on existing class structures.
* Learn how to use **inheritance** to create specialised classes that reuse and extend existing functionality.
* Explore **method overriding** and how subclasses can change or refine inherited behaviour.
* Understand the difference between **inheritance** and **composition**, and when to prefer one over the other.
* Implement small class hierarchies that model mathematical systems (like vectors and matrices).
* See how **polymorphism** allows different object types to respond to the same operation in flexible ways.

By the end, you will be able to design and extend multi-class systems that are modular, reusable, and well-structured — the kind of architecture used in real-world scientific and financial software.

##  Why Inheritance?

Different elements of the Python language can help us in different ways with one of the main challenges in programming:

* to **break big tasks up into small, manageable pieces**.

When working with objects, **inheritance** avoids code repetition by declaring a new class as a **subclass** (or a **child**)
of some other class (the **parent** class).  The child then inherits all the methods defined in the parent class.


As an **examples**, we will design a `Matrix` class, identify the overlap in functionality with the `Vector` class, and then restructure and simplify the code 
through a common parent class `Tensor`.

## Task 1. Inheritance Example

Suppose you need to process data for **students** and **teachers**.  Both have attributes in common (like a name), while differing in behavior.  We define classes, one for student objects, and one for teacher objects, as subclasses of a common parent `Person`.

In [ ]:
class Person:
    def __init__(self, first, last):
        self.firstname = first
        self.lastname = last
    def __repr__(self):
        return f"{self.firstname} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. How are you getting on?")

Then we can create a person, and make them chat ...

In [ ]:
john = Person("John", "Kelly")
print(john)
john.chat()

A `Student` is like a `Person`, except that each student **has** a student id, which can be used as email account.

In [ ]:
class Student:
    def __init__(self, first, last, number):
        self.firstname = first
        self.lastname = last
        self.student_id = number
    def __repr__(self):
        return f"{self.firstname} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. How are you getting on?")
    def email(self):
        return f"{self.student_id}@uni.ie"

As before, we now can create a student and make them chat.  We can also get their email address.

In [ ]:
anna = Student("Anna", "Byrne", 4321)
print(anna)
print(anna.email())
anna.chat()

Inheritance allows us to avoid unnnecessary repetition, by defining `Student` as a subclass of `Person`, where only the new attributes (`student_id`) and behaviour (`email`) need to be take care of.

In [ ]:
class Student(Person):
    def __init__(self, first, last, number):
        self.firstname = first
        self.lastname = last
        self.student_id = number
    def email(self):
        return f"{self.student_id}@uni.ie"

Objects this `Student` class work exactly as above.

In [ ]:
anna = Student("Anna", "Byrne", 4321)
print(anna)
print(anna.email())
anna.chat()

The `super` function allows for even more re-use of functionality from the parent class, for example in the object initialization. 

In [ ]:
class Student(Person):
    def __init__(self, first, last, number):
        super().__init__(first, last)
        self.student_id = number
    def email(self):
        return f"{self.student_id}@uni.ie"

Test it:

In [ ]:
anna = Student("Anna", "Byrne", 4321)
print(anna)

A teacher is like a person, except that a teacher has a title ...

In [ ]:
class Teacher(Person):
    def __init__(self, first, last, title):
        super().__init__(first, last)
        self.title = title
    def __repr__(self):
        return f"{self.title} {self.lastname}"
    def chat(self):
        print(f"Hi, I'm {self}. Have you done your homework?")

We can now create a teacher and make them chat:

In [ ]:
teacher = Teacher("Steven", "Kennedy", "Mr.")
teacher.chat()

There is in fact a way to make a `Teacher` chat like a normal `Person`:

In [ ]:
super(Teacher, teacher).chat()

---
**Exercises**

1. Add a method `f_last` to the `Person` class so that it returns a string of the form `"A. Byrne"` for `anna`, i.e., the first name intial followed by the last name.
How would you apply it to a `Teacher` object?  And what would it return for Mr. Kennedy?

2. Add a method `email` to the `Teacher` class so that it returns a string like `"kennedy@example.com"` for Mr. Kennedy.  How would this method affect the `Student`'s `email` method?

---

## Task 2. Matrix Algebra

Recall the `Vector` class:

In [ ]:
class Vector:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Vector({self.data})"
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __iter__(self):
        return iter(self.data)
    
    def __add__(self, other):
        assert len(self) == len(other), "length mismatch"
        return Vector([x + y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return Vector([other * x for x in self])

A **matrix** is a $2$-dimensional, rectangular collection of numerical data, like
$$
 A = \left[
  \begin{array}{ccc}
  1 & 0 & 1 \\
  2 & 1 & 1 \\
  0 & 1 & 1 \\
  1 & 1 & 2
  \end{array}
  \right].
$$
The rows of a matrix are vectors, and so are the columns. Here, the rows of $A$ are the vectors
$$
\left[
  \begin{array}{ccc}
  1 & 0 & 1 
  \end{array}
  \right],
\quad
  \left[
  \begin{array}{ccc}
  2 & 1 & 1 
  \end{array}
  \right],
\quad
  \left[
  \begin{array}{ccc}
  0 & 1 & 1 
  \end{array}
  \right],
\quad \text{and} \quad
  \left[
  \begin{array}{ccc}
   1 & 1 & 2
  \end{array}
  \right].
$$
Matrices are **like vectors**: you can add two matrices, provided they have the **same shape** (i.e., the same number of rows and the same number of columns), and you can stretch or shrink a matrix by a scalar factor.  So we should expect the methods implementing such behaviour for matrices to be very similar to the corresponding methods for vectors.  In fact, by regarding a matrix as a **list of vectors** (its rows), just like a vector is a **list of numbers**, the matrix methods can be almost identical to the vector methods.  And we can take advantage of the fact that python already knows how to compute with `Vector`s ...

In [ ]:
class Matrix:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Matrix({self.data})"
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __iter__(self):
        return iter(self.data)
    
    def __add__(self, other):
        assert len(self) == len(other), "length mismatch"
        return Matrix([x + y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return Matrix([other * x for x in self])

Now we can create a `Matrix` object corresponding to the matrix $A$ above, as follows.

In [ ]:
ma = Matrix([
    Vector([1, 0, 1]),
    Vector([2, 1, 1]),
    Vector([0, 1, 1]),
    Vector([1, 1, 2])
])
ma

Compute $3A$, the $3$-multiple of $A$:

In [ ]:
3 * ma

Create another matrix, $B$, of the same  size as $A$ and compute the sume $A + B$:

In [ ]:
mb = Matrix([
    Vector([1, 2, 3]),
    Vector([4, 5, 6]),
    Vector([0, 0, 0]),
    Vector([7, 8, 9])
])
ma + mb

---
**Exercises**

1. What happens when you try and add a square $4 \times 4$-matrix to the matrix `ma`? 
1. Add a method `__eq__` to both the `Vector` and the `Matrix` class, so that `v == w` returns `True` if vectors (or matrices) `v` and `w`
have exactly the same entries in corresponding positions.


---

## Task 3. `Tensor`, a common parent class

There is a lot of similarity between `Vector` and `Matrix`.  Moving common code into a common parent class, `Tensor`, can avoid a lot of repetition. 

In [ ]:
class Tensor:
    """a blueprint class for vectors and matrices"""

    # constructor
    def __init__(self, data):
        self.data = data

   # list like behaviour
    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return iter(self.data)

    def __getitem__(self, i):
        return self.data[i]

class Vector(Tensor):
    def __repr__(self):
        return f"Vector({self.data})"
    
    def __add__(self, other):
        assert len(self) == len(other), "length mismatch"
        return Vector([x + y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return Vector([other * x for x in self])
    
class Matrix(Tensor):
    def __repr__(self):
        return f"Matrix({self.data})"
    
    def __add__(self, other):
        assert len(self) == len(other), "length mismatch"
        return Matrix([x + y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return Matrix([other * x for x in self])

Check that all works as before:

In [ ]:
ma = Matrix([Vector([1, 0, 1]), Vector([2, 1, 1]), Vector([0, 1, 1]), Vector([1, 1, 2])])
mb = Matrix([Vector([1, 2, 3]), Vector([4, 5, 6]), Vector([0, 0, 0]), Vector([7, 8, 9])])
print(3 * ma)
print(ma + mb)

---
**Exercises**

1. Reformulate `__eq__` as a method in the `Tensor` class, so that it does the right thing for both `Vector` and `Matrix`.
2. What happens now if you try and add a square $4 \times 4$-matrix to `ma`?

---

## Task 4. Types

Each Python object has a **type**.  We can get the type of object `obj` by calling
```python
   type(obj)
```
If `obj` is an integer then its type is `int`.
If `obj` is an instance of a class `C` then its type is `C`.
The type `t` itself is an object, with attributes and behaviour. E.g.,
```python
   t.__name__
```
is the name of type `t`, as a string.  And `t` can be called, like a function, for type conversion, or in the case of a class, to create new objects.

In [ ]:
t = type(3)
print(t)
print(t(3.14))

We can use this kind of introspection in the `Vector` and `Matrix` classes to avoid even more repetition.  
More precisely, in the parent class `Tensor`, we can define a common method
```python
    def __repr__(self):
        return f"{type(self).__name__}({self.data})"
```
for the string representation, which works for both `Vector` and `Matrix`.  And we can define common methods for addition and scalar multiplication, like:
```python
    def __rmul__(self, other):
        return type(self)([other * x for x in self])
```
The simplified class hierarchy then looks as follows.

In [ ]:
class Tensor:
    """a blueprint class for vectors and matrices"""

    # constructor
    def __init__(self, data):
        self.data = data

   # list like behaviour
    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return iter(self.data)

    def __getitem__(self, i):
        return self.data[i]

    def __repr__(self):
        return f"{type(self).__name__}({self.data})"
    
    def __add__(self, other):
        assert len(self) == len(other), "length mismatch"
        return type(self)([x + y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return type(self)([other * x for x in self])

class Vector(Tensor):
    pass

class Matrix(Tensor):
    pass

Here, `pass` is Python's "do nothing" command.  Except for the type there is currently no difference between `Vector` and `Matrix` ...

In [ ]:
ma = Matrix([Vector([1, 0, 1]), Vector([2, 1, 1]), Vector([0, 1, 1]), Vector([1, 1, 2])])
mb = Matrix([Vector([1, 2, 3]), Vector([4, 5, 6]), Vector([0, 0, 0]), Vector([7, 8, 9])])
print(3 * ma)
print(ma + mb)

We'll add the `dot` method to `Vector` and a transpose method to `Matrix` next.

---
**Exercises**

1. The "dot product" of two vectors $v$ and $w$ of the same length is the number $v.w = \sum_i v_i w_i$, i.e., the sum pof the products of corresponding entries in $v$ and $w$.  Write a method `dot(self, other)` that computes this number.

1. Which other methods can you imagine that would apply only to vectors, or only to matrices?

---

## Task 5.  List unpacking

Some functions, like `print`, expect and handle an arbitrary number of arguments: the command
```python
print("a", "b", "c")
```
will print the letters `a`, `b` and `c`, separated by spaces.  But:

In [ ]:
l = ["a", "b", "c"]
print(l)

has a different effect.  To get the letters printed as before, you'd need to get rid of the brackets around `"a", "b", "c"` ...
That's exactly what the **list unpacking operator** `*` does, like so:

In [ ]:
print(*l)

`zip` is another example of such a function.

In [ ]:
print(list(zip(l)))
print(list(zip(*l)))

The **transpose**  $A^T$ of a matrix $A$ has the columns of $A$ as its rows. E.g.,
$$
 A = \left[
  \begin{array}{ccc}
  1 & 0 & 1 \\
  2 & 1 & 1 \\
  0 & 1 & 1 \\
  1 & 1 & 2
  \end{array}
  \right],\qquad
  A^T = \left[
  \begin{array}{cccc}
  1 & 2 & 0 & 1 \\
  0 & 1 & 1 & 1 \\
  1 & 1 & 1 & 2
  \end{array}
  \right].
$$

In principle, `zip` applied to the rows of $A$ yields the columns of $A$.
So we can use `zip` to compute the transpose $A^T$.
But for this to work, we need to provide the rows of $A$ as individual arguments to `zip`.
Luckily, list unpacking works for `Vector` and `Matrix` objects.

In [ ]:
print(*ma)

In [ ]:
for row in ma:
    print(*row)

Here is a function `transpose` that computes the transpose $A^T$ of a matrix $A$, using `zip` with list unpacking.

In [ ]:
def transpose(matrix):
    return Matrix([Vector(x) for x in zip(*matrix)])

In [ ]:
mt = transpose(ma)
for row in mt:
    print(*row)

---
**Exercises**

1. A kind of opposite to list unpacking is the `*args` convention in function definition headers that allows us to **define* functions with arbitrarily many arguments.
Look up its documentation.

2. Can an `*args` argument in the `Vector` initialization be used to create vector objects as `v = Vector(1, 2, 3)`, i.e. without the square brackets?

3. If vectors are created in this way, would it make list unpacking necessary in other places?

---

## Task 6. Decorators

Rather than a **function** `transpose`, we'd like to have a **method**, `T` say, for computing the transpose of `ma` as `ma.T()`.
This we can define, inside the class `Matrix`, as follows:
```python
    def T(self):
        return Matrix([Vector(x) for x in zip(*self)])
```
But, given that the transpose of a matrix can be seen as an attribute of the matrix, it would feel more natural to simply use `ma.T` (without the pair of parentheses) for the transpose of `ma`.  This can be achieved by using a decorator like this:
```python
    @property
    def T(self):
        return Matrix([Vector(x) for x in zip(*self)])
```
There are other decorators that can be used to modify the bahaviour of a function. We might see some examples later.

For now, here is the final version of the `Tensor` class hierarchy, with a `dot` method for dot products of `Vector`s and a property `T` for `Matrix` objects.

In [ ]:
class Vector(Tensor):
    def dot(self, other):  # v . v
        assert len(self) == len(other), "length mismatch"
        return sum(x * y for x, y in zip(self, other))

class Matrix(Tensor):
    @property
    def T(self):
        return Matrix([Vector(x) for x in zip(*self)])

With this we can compute dot products of vectors and matrix transposes.

In [ ]:
ma = Matrix([Vector([1, 0, 1]), Vector([2, 1, 1]), Vector([0, 1, 1]), Vector([1, 1, 2])])
v, w = ma[:2]
print(v, w)
print(v.dot(w))
print(ma.T)

---
**Exercises**

1. Using the `dot` methods of the `Vector` class, define a **property** `squared_length` that returns the dot product $v.v$ of a vector $v$ with itself.

2. Write a `__matmul__(self, other)` method for the `Vector` class that uses the `dot` method and the `T` property to compute the product $v \cdot m$ of
a vector $v$ and a matrix $m$.  Which Python operator gets overloaded by `__matmul__`?

3. Write a `__matmul__`(self, other)` method for the `Matrix` class that refers to the method of the same name in the `Vector` class, to compute tha product $AB$ of two matrices $A$ and $B$ of suitable shapes.
---

## Task 7. Files, Modules, `import`

Writing the code for `Tensor`, `Vector` and `Matrix` into a file, say `tensor.py` makes this file a Python **module** named `tensor`.

In a new Python session, notebook or module, we don't need to type this code again, but can simply import the module, or elements from it, with commands like
```python
    import tensor
    from tensor import Tensor, Vector, Matrix
    from tensor import *
```
With the first version, the `Vector` class is known as `tensor.Vector` in the session.  With the other versions, it can be used as `Vector`.

In [ ]:
from tensor import *
v = Vector([1,2,3])
v

---
**Exercises**

1. Write the people hierachy of `Person`, `Student` and `Teacher` into a file `people.py`.

2. How would you load thes class definitions into a new Python session?
---

## Summary

In this notebook, you explored how to build richer, more flexible systems using **advanced object-oriented programming (OOP)** concepts:

* You learned how **inheritance** allows one class to extend another, reducing repetition and improving maintainability.
* You saw how **method overriding** enables subclasses to modify or extend inherited behaviour to suit specialised needs.
* You encountered **polymorphism**, where different objects can respond to the same operation in context-appropriate ways.
* You learned about further useful Python concepts, like **types**, **list unpacking**, **decorators** and **modules**. 

Together, these techniques let you organise complex problems — such as vector spaces, portfolios, or physical systems — into coherent, modular structures.
